1. Sistemas Lineares - Decomposição LU

In [ ]:
import numpy as np
import time

def func1(A):
    """
    Decomposição de uma matriz A (quadrada) nos fatores L e U (LU sem pivotamento).
    Implementação direta para entender melhor os passos clássicos da decomposição.
    """
    n = A.shape[0]
    U = A.copy()
    L = np.eye(n)
    for j in range(n):
        for i in range(j+1, n):
            L[i, j] = U[i, j] / U[j, j]
            U[i, j:] = U[i, j:] - L[i, j] * U[j, j:]
    return L, U

def func2(A, p):
    """
    Versão modificada da decomposição LU com "faixa de banda" p.
    Faz o cálculo olhando só para a banda (e não a matriz inteira).
    Útil para matrizes grandes e esparsas, por exemplo.
    """
    n = A.shape[0]
    U = A.copy()
    L = np.eye(n)
    for j in range(n):
        v = min(n, j+p+1)
        for i in range(j+1, v):
            L[i, j] = U[i, j] / U[j, j]
            U[i, j:v] = U[i, j:v] - L[i, j] * U[j, j:v]
    return L, U

# Teste e comparação dos métodos propostos:
n = 2000
p = 2
A = np.zeros((n, n))
for i in range(n):
    for j in range(max(0, i-p), min(n, i+p+1)):
        A[i, j] = np.random.normal()

start_time = time.time()
L1, U1 = func1(A)
print("Tempo func1:", time.time() - start_time)

start_time = time.time()
L2, U2 = func2(A, p)
print("Tempo func2:", time.time() - start_time)

print("Norma func1:", np.linalg.norm(L1 @ U1 - A))
print("Norma func2:", np.linalg.norm(L2 @ U2 - A))


2. Zeros de funcões - Bissecção de Newton

In [ ]:
def bissecao(f, a, b, tol=1e-8):
    """
    Algoritmo de bisseção para encontrar raiz de função contínua f.
    Vai cortando o intervalo ao meio até achar a raiz com precisão tol.
    """
    fa, fb = f(a), f(b)
    if fa * fb > 0:
        raise ValueError("Intervalo não contém raiz!")
    while abs(b-a) > tol:
        c = (a + b) / 2.0
        fc = f(c)
        if fa * fc < 0:
            b, fb = c, fc
        else:
            a, fa = c, fc
    return (a + b) / 2.0


def newton(f, df, x0, tol=1e-8, nmax=500):
    """
    Método de Newton clássico. 
    df = derivada da função f.
    Vai iterando a partir de x0 até achar a raiz.
    """
    x = x0
    for _ in range(nmax):
        fx = f(x)
        dfx = df(x)
        if abs(fx) < tol:
            return x
        if dfx == 0:
            raise ValueError("Derivada zerou! Não dá pra seguir.")
        x = x - fx/dfx
    raise RuntimeError("Não convergiu no máximo de iterações.")

# Testando:


def f(x):
    return x**2 - 2


def df(x):
    return 2*x


raiz_bissecao = bissecao(f, 0, 2)
raiz_newton = newton(f, df, 1.5)
print("Raiz por bisseção:", raiz_bissecao)
print("Raiz por Newton:", raiz_newton)

3. Decomposição SVD e Francis

In [ ]:
def svd_manual(A):
    """
    SVD feita usando comandos do numpy, para facilitar.
    Não implementei a partir do zero porque ia ficar enorme.
    """
    U, S, VT = np.linalg.svd(A)
    return U, S, VT


# Teste:
np.random.seed(42)
A = np.random.randn(5, 4)  # matriz de teste, 5 linhas x 4 colunas
U, S, VT = svd_manual(A)

# Reconstrução correta:
# Usa só as 4 primeiras colunas de U (mesmo número que colunas de A)
reconstruida = U[:, :4] @ np.diag(S) @ VT
print("Reconstrução SVD:", np.allclose(reconstruida, A))

4. Interpolação Polinomial

In [ ]:
def interpolacao_lagrange(x, y, xp):
    """
    Interpolação polinomial de Lagrange. 
    Recebe pontos (x, y), retorna valor interpolado em xp.
    """
    n = len(x)
    yp = 0
    for i in range(n):
        L = 1
        for j in range(n):
            if i != j:
                L *= (xp - x[j]) / (x[i] - x[j])
        yp += y[i] * L
    return yp


# Teste:
x = [0, 1, 2]
y = [1, 2, 0]
print("Valor interpolado (x=1.5):", interpolacao_lagrange(x, y, 1.5))

5. Mínimos Quadrados

In [ ]:
def minimos_quadrados(x, y):
    """
    Ajuste de mínimos quadrados via fórmula normal, para reta.
    """
    X = np.vstack([np.ones(len(x)), x]).T
    beta = np.linalg.inv(X.T @ X) @ X.T @ y
    return beta


# Teste:
x = np.array([0, 1, 2, 3])
y = np.array([1, 2, 1.3, 3.75])
b0, b1 = minimos_quadrados(x, y)
print(f"Ajuste: y = {b0:.2f} + {b1:.2f} x")